In [1]:
import serial
from tqdm.notebook import tqdm, trange
import pandas as pd
import numpy as np
import torch as pt

In [2]:
def coef_to_list(coefs):
    coef_idx = []
    coef_val = []
    for idx, val in coefs.items():
        coef_idx.append(idx)
        coef_val.append(val)
    return (coef_idx, coef_val)

def grad_calc(loss, coefs):
    coef_idx, coef_val = coef_to_list(coefs)
    grad_val = pt.autograd.grad(loss, coef_val)
    gradient = {idx:val for idx, val in zip(coef_idx, grad_val)}
    return gradient

In [3]:
with serial.Serial('/dev/ttyUSB0', 115200) as ser:
    serial_lines = [ser.readline() for _ in trange(1200)]
serial_lines = pd.DataFrame([ser.decode("utf-8").strip().split(',') for ser in serial_lines], columns=['mag_x', 'mag_y', 'mag_z']).astype(float)
earth_mag = np.array([[17848.0, 4593.9, 50836.6]]).T / 1000

In [44]:
serial_mag_t = pt.from_numpy(serial_lines.values).type(pt.Tensor)
data_scale = pt.sqrt((serial_mag_t**2).sum(dim=1))
earth_mag_t = pt.from_numpy(earth_mag).type(pt.Tensor)

In [45]:
coefs = {
    'bias': pt.zeros(3),
    'skew': pt.eye(3),
}

In [48]:
alpha = 0.8
beta = 0.9
grad = {}
grad_sq = {}
clamp = 1e-16
learn_rate = 1e-3

In [49]:
loss_rcd = []
t = trange(1000)
for i in t:
    for val in coefs.values(): val.requires_grad = True

    earth_adj = (coefs['skew'] @ earth_mag_t) + coefs['bias']
    earth_scale = pt.sqrt((earth_adj**2).sum())
    error = data_scale - earth_scale
    loss = pt.sqrt((error**2).mean())
    
    loss_rcd.append(loss.detach().cpu().numpy())
    t.set_postfix({'loss': loss_rcd[-1]})

    gradient = grad_calc(loss, coefs)
    with pt.no_grad():
        for idx in gradient.keys():
            grad[idx] = (alpha * gradient[idx]) + ((1-alpha) * grad.get(idx, gradient[idx]))
            grad_sq[idx] = (beta * (gradient[idx]**2)) + ((1-beta) * grad_sq.get(idx, gradient[idx]**2))
            step = grad[idx] / pt.clamp(pt.sqrt(grad_sq[idx]), clamp, np.inf)
            coefs[idx] = coefs[idx] - (step * learn_rate)

In [57]:
results = (coefs['skew'] @ serial_mag_t.t()).t() + coefs['bias']

In [61]:
pd.DataFrame(results.numpy()).min()

0   -53.908890
1   -58.291321
2   -41.097675
dtype: float32